In [49]:
!kaggle datasets download -d notshrirang/spotify-million-song-dataset

Dataset URL: https://www.kaggle.com/datasets/notshrirang/spotify-million-song-dataset
License(s): CC0-1.0
spotify-million-song-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [50]:
!unzip -q spotify-million-song-dataset.zip

replace spotify_millsongdata.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [51]:
import pandas as pd

In [52]:
df = pd.read_csv('/content/spotify_millsongdata.csv')

In [53]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [54]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [57]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [58]:
df.shape

(5000, 3)

In [59]:
df.isnull().sum()

,0
artist,0
song,0
text,0


In [60]:
df =df.sample(5000)

In [61]:
df.head(10)

,artist,song,text
1308,Allman Brothers Band,Try It One More Time,Well I just can't believe it's time for leavin...
679,Conway Twitty,Bad Seed My Daddy Sowed,I was found in Wilson County just a mile outsi...
4621,Backstreet Boys,That's The Way I Like It,Hey yeah \r\n \r\nThat's the way I like it ...
1956,Phil Collins,Wear My Hat,She came looking for me with her arms open wid...
4998,Hollies,Everyday,"Every day, it's a-getting closer \r\nGoing fa..."
1025,Lloyd Cole,Love You So What,You up and leave me cold turkey \r\nWhen you ...
4100,Iron Maiden,Churchill Speech,We shall go on to the end. \r\nWe shall fight...
868,Etta James,Lie No Better,Here you come draggin' in \r\nThree a.m. agai...
3460,Jimmy Buffett,Dreamsicle,"Let me tell you now he sips, he quips \r\nDre..."
2397,Bruce Springsteen,Don't Look Back,Cold rain running down the front of my shirt ...


In [62]:
df.shape

(5000, 3)

In [66]:
df['text'][0]

"she lost her focus yesterday  \r she lost her focus in the sun  \r she sits and stares at it for hours  \r she says she likes to go outside  \r she gets hung up on the wind  \r she gets lost inside the happy noise  \r   \r sometimes i don't understand her  \r sometimes i don't want to  \r understand her  \r   \r she says no no no no  \r she don't want no double life  \r she says i should know why  \r she seems happier at night  \r her color tv and her chemical smile  \r   \r i don't want to know the reason  \r i don't want to know the reason why  \r   \r she says  \r jesus owes her money  \r she says  \r that the angels are her friends  \r   \r what in the hell does that mean?  \r   \r she's got the chemical smile\r \r "

In [67]:
#Text Cleaning/ Text Preprocessing

In [68]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [69]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [70]:
import nltk
nltk.download('punkt')
df['text'] = df['text'].apply(lambda x: tokenization(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [73]:
similarity[0]

array([1.        , 0.08157625, 0.02040917, ..., 0.02161844, 0.01058605,
       0.06288452])

In [74]:
df[df['song'] == 'Crying Over You']

,artist,song,text
4924,UB40,Crying Over You,cri over you in the morn cri over you in the e...
4714,ABBA,Crying Over You,i 'm waitin ' for you babi i 'm sit all alon i...


In [79]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [80]:
recommendation('Crying Over You')

['Long Tall Glasses',
 'Do You Want To Dance',
 'Ten Guitars',
 'All She Wants To Do Is Dance',
 'Jupiter Spin',
 'Again And Again',
 'Mind Drive',
 'Dancing In The Dark',
 'Dance, Dance',
 "Don't Wanna Dance Alone",
 'I Could Sing Of Your Love Forever',
 'Choices',
 'This Is Your Time',
 "Your Mama Don't Dance",
 'Dancing With The Big Boys',
 'For Whom The Bell Tolls',
 'It Only Happens When I Dance With You',
 'Buffalo Gals',
 'Dancing With The Mountains',
 'Happy Song']

In [88]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 9.6 MB/s eta 0:00:00


In [127]:
with open('app.py', 'w') as f:
    f.write('''
import streamlit as st
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set Streamlit page config for better UI
st.set_page_config(page_title="Music Recommender System", page_icon="ᯓᡣ𐭩", layout="wide")

# Apply custom CSS for aesthetics and layout
st.markdown(
    """
    <style>
    .main {
        background-color: #1e1e1e;  /* Dark gray/black background */
        color: white;
    }
    .stButton button {
        background-color: #4CAF50;
        color: white;
        border-radius: 5px;
        font-size: 16px;
        padding: 8px 16px;
        margin-top: 15px;
    }
    .stHeader {
        text-align: center;
        font-size: 28px;
        color:white;
        margin-bottom: 20px;
    }
    .stImage {
        display: block;
        margin-left: auto;
        margin-right: auto;
        width: 100px;  /* Small logo size */
        margin-bottom: 20px;  /* Spacing between logo and rest of the content */
    }
    .stSearchBox {
        text-align: center;
        margin-bottom: 20px;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Set Spotify credentials
CLIENT_ID = "05f32cadca4549949667a0c52a83c082"
CLIENT_SECRET = "c18e26f41e8b4ff9b1b56af2bb2fdbd2"

# Initialize the Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_song_album_cover_url(song_name, artist_name):
    search_query = f"track:{song_name} artist:{artist_name}"
    results = sp.search(q=search_query, type="track")

    if results and results["tracks"]["items"]:
        track = results["tracks"]["items"][0]
        album_cover_url = track["album"]["images"][0]["url"]
        return album_cover_url
    else:
        return "https://i.postimg.cc/0QNxYz4V/social.png"  # Fallback image

def recommend(song_name):
    search_query = f"track:{song_name}"
    results = sp.search(q=search_query, type="track")

    recommended_music_names = []
    recommended_music_posters = []

    if results and results['tracks']['items']:
        # Fetch similar tracks (limit 5)
        tracks = results['tracks']['items'][:5]
        for track in tracks:
            recommended_music_names.append(track['name'])
            recommended_music_posters.append(track['album']['images'][0]['url'])
    else:
        st.warning("No recommendations found. Please try another song.")

    return recommended_music_names, recommended_music_posters

# Streamlit app interface
st.markdown('<h1 class="stHeader"> 🎧  Music Recommender System.. ๋࣭ ⭑⚝  </h1>', unsafe_allow_html=True)

# Spotify logo centered and small
#st.image("/content/spotify.jpeg", use_column_width=False, width=100)

# Single input box for song search
song_name = st.text_input('Enter a song name:', '')

# Show recommendation button
if st.button('Show Recommendations') and song_name:
    recommended_music_names, recommended_music_posters = recommend(song_name)

    # Display recommendations in a grid
    cols = st.columns(5)  # 5 columns for 5 recommended songs
    for i in range(5):
        if i < len(recommended_music_names):
            with cols[i]:
                st.image(recommended_music_posters[i], use_column_width=True)
                st.markdown(f"**{recommended_music_names[i]}**", unsafe_allow_html=True)

# Footer
st.markdown("""
    <hr>
    <p style="text-align: center; font-size: 14px;">Powered by Spotify API and Streamlit | Developed by Srikanth ᯓᡣ𐭩 </p>
    """, unsafe_allow_html=True)
''')


In [128]:
!wget -q -O - ipv4.icanhazip.com

34.19.85.129


In [ ]:

!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.19.85.129:8501

your url is: https://rich-bottles-kneel.loca.lt
